# 1. 把底模safetensors转化为文件夹权重模式

在这里下载底模https://civitai.com/models/27259?modelVersionId=221220然后移动到文件夹

In [1]:
!wget --content-disposition "https://civitai.com/api/download/models/221220?type=Model&format=SafeTensor&size=pruned&fp=fp16"

'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [ ]:
from diffusers import StableDiffusionPipeline

checkpoint_path = r"tmndMix_tmndMixSPRAINBOW.safetensors"
save_path = r"basemodel"
# 加载 .safetensors 文件
pipeline = StableDiffusionPipeline.from_single_file(checkpoint_path)

# 将模型保存为 diffusers 格式
pipeline.save_pretrained(save_path)

# 2. 把数据集解压

数据集要的所有东西全部放在共享文件夹了（images）  

In [ ]:
!unzip images/image_resized.zip
!cp images/merged_joints2d.txt image_resized/merged_joints2d.txt
!cp images/filtered_camera_params.txt image_resized/camera_params.txt
!cp images/image_features.txt image_resized/image_features.txt
!cp images/image_smpl_normal.txt image_resized/image_smpl.txt

In [ ]:
!unzip images/image_mirror_resized.zip
!cp images/mirror_merged_joints2d.txt image_resized/merged_joints2d.txt
!cp images/mirror_filtered_camera_params.txt image_mirror_resized/camera_params.txt
!cp images/image_features.txt image_mirror_resized/image_features.txt

# 3. 训练

In [ ]:
!git clone https://github.com/sodabreak/PoseCtrl.git
!cd poseCtrl

单卡

In [ ]:
python train_colab_V7_AllData.py --pretrained_model_name_or_path "/basemodel" --base_point_path "/PoseCtrl/dataSet/standardVertex_2.txt" --data_root_path_2 "/image_resized" --data_root_path_3 "/image_mirror_resized" --train_batch_size 4 --save_steps 2000 

多卡

- 先运行配置
- 回答问题：使用混合精度fp16

In [ ]:
accelerate config

In [ ]:
accelerate launch --num_processes 2 --multi_gpu --mixed_precision "fp16" train_colab_V6_AllData.py --pretrained_model_name_or_path "/basemodel" --base_point_path "/PoseCtrl/dataSet/standardVertex_2.txt" --data_root_path_2 "/image_resized" --data_root_path_3 "/image_mirror_resized" --train_batch_size 4 --save_steps 2000 

# 权重处理

所有训练得到的权重都在/sd-pose_ctrl文件夹内



# inference

In [ ]:
""" V6 """
import torch
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoencoderKL
from PIL import Image
import sys
import os
current_dir = 'content'

sys.path.append('/content/PoseCtrl')
sys.path.append('/content/PoseCtrl/poseCtrl')
from poseCtrl.models.pose_adaptor import VPmatrixPoints, ImageProjModel
from poseCtrl.models.attention_processor import AttnProcessor, PoseAttnProcessor
from poseCtrl.data.dataset import CombinedDataset, load_base_points
from poseCtrl.models.posectrl import PoseCtrl,PoseCtrlV1,PoseCtrlV5
import numpy as np
from poseCtrl.pipelines.Pose_pipelines import PoseControlNet
from poseCtrl.pipelines.Pose_pipelines import PoseControlNet, PoseControlNetV6

base_point_path=r'/content/drive/MyDrive/PoseCtrl/dataSet/standardVertex_2.txt'
raw_base_points=load_base_points(base_point_path)  
base_model_path = r"/content/drive/MyDrive/basemodel"
vae_model_path = "stabilityai/sd-vae-ft-mse"
image_encoder_path = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
ip_ckpt = r"/content/drive/MyDrive/ckpt/points_posectrl2000.bin"
device = "cuda"
def image_grid(imgs_np: np.ndarray, rows: int, cols: int):
    """
    imgs_np: np.ndarray of shape (B, H, W, C) and values in [0, 1] (float)
    rows, cols: number of rows and columns in grid
    """
    assert imgs_np.ndim == 4 and imgs_np.shape[0] == rows * cols, "Input shape must be (B, H, W, C)"
    B, H, W, C = imgs_np.shape

    # 归一化 -> uint8 -> PIL
    pil_imgs = [Image.fromarray((img * 255).astype(np.uint8)) for img in imgs_np]

    # 创建大图
    grid_img = Image.new('RGB', size=(cols * W, rows * H))
    
    for idx, img in enumerate(pil_imgs):
        x = (idx % cols) * W
        y = (idx // cols) * H
        grid_img.paste(img, box=(x, y))

    return grid_img

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)

# load SD pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    feature_extractor=None,
    safety_checker=None
)
val_dataset = CombinedDataset(
        # path1=args.data_root_path_1,
        path2="/content/drive/MyDrive/images_01/test",
        tokenizer=pipe.tokenizer,
    )


data = val_dataset[0]

from torchvision import transforms

transform = transforms.Resize((256, 256))

""" denormalize 是因为dataloader的图片加载完之后会被transform，正常的图片只要是PIL就行 """
def denormalize(tensor, mean, std):
    return tensor * std + mean

mean = 0.5
std = 0.5


image = data['image']
points = data['joints_image']
image = denormalize(image, mean, std)
image_pil = transforms.ToPILImage()(image) 
points_pil = transforms.ToPILImage()(points)
points = data['joints_image'].to(torch.float16).unsqueeze(0).to(device)
vmatrix = data['view_matrix'].to(torch.float16).unsqueeze(0).to(device)
pmatrix = data['projection_matrix'].to(torch.float16).unsqueeze(0).to(device)
text = 'highly detailed, anime, 1girl, blue_eyes, purple long_hair, black dress, smile, white simple background'
pose_model = PoseControlNetV6(pipe, image_encoder_path, ip_ckpt, raw_base_points, device,8)
images = pose_model.generate( prompt = text, num_samples=4, num_inference_steps=50, seed=42, V_matrix=vmatrix, P_matrix=pmatrix, points=points)
grid = image_grid(images, 1, 4)
combined_image = Image.new('RGB', (6*image_pil.width, image_pil.height))
combined_image.paste(image_pil, (0, 0))
combined_image.paste(points_pil, (image_pil.width, 0))
combined_image.paste(grid, (2*image_pil.width, 0))
combined_image

In [ ]:
""" V7 """
import torch
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoencoderKL
from PIL import Image
import sys
import os
current_dir = r'F:\Projects\diffusers\Project'
current_dir = os.path.join(current_dir, "PoseCtrl")
sys.path.append(current_dir)
sys.path.append(os.path.join(current_dir,"poseCtrl"))
from poseCtrl.models.pose_adaptor import VPmatrixPoints, ImageProjModel
from poseCtrl.models.attention_processor import AttnProcessor, PoseAttnProcessor
from poseCtrl.data.dataset import CombinedDataset, load_base_points, CombinedDatasetTest
from poseCtrl.models.posectrl import PoseCtrl,PoseCtrlV1,PoseCtrlV5
import numpy as np
from poseCtrl.pipelines.Pose_pipelines import PoseControlNet
from poseCtrl.pipelines.Pose_pipelines import PoseControlNet, PoseControlNetV6, PoseControlNetV7
base_point_path=r"F:\Projects\diffusers\Project\PoseCtrl\dataSet\standardVertex_2.txt"
raw_base_points=load_base_points(base_point_path)  

base_model_path = r"F:\Projects\diffusers\ProgramData\basemodel"
# vae_model_path = "stabilityai/sd-vae-ft-mse"
image_encoder_path = r"C:\Users\30631\.cache\huggingface\hub\models--laion--CLIP-ViT-H-14-laion2B-s32B-b79K\snapshots\1c2b8495b28150b8a4922ee1c8edee224c284c0c"
ip_ckpt = r"F:\Projects\diffusers\Backup\ckpt\posectrl.bin"
device = "cuda"
def image_grid(imgs_np: np.ndarray, rows: int, cols: int):
    """
    imgs_np: np.ndarray of shape (B, H, W, C) and values in [0, 1] (float)
    rows, cols: number of rows and columns in grid
    """
    assert imgs_np.ndim == 4 and imgs_np.shape[0] == rows * cols, "Input shape must be (B, H, W, C)"
    B, H, W, C = imgs_np.shape

    # 归一化 -> uint8 -> PIL
    pil_imgs = [Image.fromarray((img * 255).astype(np.uint8)) for img in imgs_np]

    # 创建大图
    grid_img = Image.new('RGB', size=(cols * W, rows * H))
    
    for idx, img in enumerate(pil_imgs):
        x = (idx % cols) * W
        y = (idx // cols) * H
        grid_img.paste(img, box=(x, y))

    return grid_img

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
# vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)

# load SD pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    feature_extractor=None,
    safety_checker=None
)

val_dataset = CombinedDatasetTest(
        # path1=args.data_root_path_1,
        path2=r"F:\\Projects\\diffusers\\ProgramData\\test",
        # path3=args.data_root_path_3,
        # path4=args.data_root_path_4,
        # path5=args.data_root_path_5,
        tokenizer=pipe.tokenizer,
        txt_subdir_name="F:\\Projects\\diffusers\\ProgramData\\new_data\\image\\smpl"
    )

data = val_dataset[0]

from torchvision import transforms

transform = transforms.Resize((256, 256))

""" denormalize 是因为dataloader的图片加载完之后会被transform，正常的图片只要是PIL就行 """
def denormalize(tensor, mean, std):
    return tensor * std + mean

mean = 0.5
std = 0.5


image = data['image']
points = data['joints_image']
image = denormalize(image, mean, std)
image_pil = transforms.ToPILImage()(image) 
points_pil = transforms.ToPILImage()(points)
# points = data['joints_image'].to(torch.float16).unsqueeze(0).to(device)
points = data['points'].to(torch.float16).unsqueeze(0).to(device)
vmatrix = data['view_matrix'].to(torch.float16).unsqueeze(0).to(device)
pmatrix = data['projection_matrix'].to(torch.float16).unsqueeze(0).to(device)
# text = 'highly detailed, anime, 1girl, blue_eyes, purple long_hair, black dress, smile, white simple background'
text = data['text']

pose_model = PoseControlNetV7(pipe, image_encoder_path, ip_ckpt, raw_base_points, device)

images = pose_model.generate( prompt = text, num_samples=4, num_inference_steps=50, seed=42, V_matrix=vmatrix, P_matrix=pmatrix, points=points)
grid = image_grid(images, 1, 4)
combined_image = Image.new('RGB', (6*image_pil.width, image_pil.height))
combined_image.paste(image_pil, (0, 0))
combined_image.paste(points_pil, (image_pil.width, 0))
combined_image.paste(grid, (2*image_pil.width, 0))
combined_image

# data preprocess

In [ ]:
""" 1. 把文件夹的selected_joints2d变成一个文件merged_joints2d"""
import os

def merge_joints2d(root_dir, output_path):
    with open(output_path, 'w', encoding='utf-8') as out_file:
        for folder_name in os.listdir(root_dir):
            folder_path = os.path.join(root_dir, folder_name)
            if not os.path.isdir(folder_path):
                continue

            joints_path = os.path.join(folder_path, 'selected_joints2d.txt')
            if os.path.exists(joints_path):
                out_file.write(f"# {folder_name}\n")  # 写入图像文件名
                with open(joints_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        out_file.write(line.strip() + '\n')
                out_file.write('\n')  # 每组之间加空行

# 使用示例
root_folder = r"F:\Projects\diffusers\ProgramData\points\output_2\output"
output_txt = r"F:\Projects\diffusers\ProgramData\points\output_2\output/merged_joints2d.txt"
merge_joints2d(root_folder, output_txt)

In [ ]:
"""  把param.py处理成smpl参数的一个文件image_smpl.txt """
import os
import re
import ast
from typing import Dict, List, Tuple, Optional

_VAR_RE = re.compile(r'^\s*([^=\s]+)\s*=')


def _extract_first_bracket_block(s: str) -> Tuple[str, int, int]:
    """
    从字符串 s 中提取第一个以 '[' 开始、匹配到的 ']' 结束的片段。
    返回 (片段字符串, start_idx, end_idx)；找不到或不匹配则抛错。
    """
    start = s.find('[')
    if start == -1:
        raise ValueError("No '[' found.")
    depth = 0
    for i in range(start, len(s)):
        c = s[i]
        if c == '[':
            depth += 1
        elif c == ']':
            depth -= 1
            if depth == 0:
                return s[start:i+1], start, i+1
    raise ValueError("Unmatched '[' ']' in string.")

def _reshape_to_21x3(arr) -> List[List[float]]:
    """
    接受 literal_eval 后的列表(可一维或二维)，返回严格 21x3 的二维 list[21][3]。
    """
    # 若是二维：铺平
    if isinstance(arr, list) and arr and isinstance(arr[0], (list, tuple)):
        flat = [float(x) for row in arr for x in row]
    elif isinstance(arr, list):
        flat = [float(x) for x in arr]
    else:
        raise ValueError(f"Unsupported parsed type: {type(arr)}")

    if len(flat) != 63:
        raise ValueError(f"Total elements = {len(flat)}; expected 63 for 21x3.")

    return [flat[i:i+3] for i in range(0, 63, 3)]

def tensors_to_block_txt(
    input_path: str,
    output_txt_path: str,
    names_filter: Optional[List[str]] = None,
    overwrite: bool = True
) -> Dict[str, object]:
    """
    读取 input_path（如 synthesis_param.py），将每个赋值的 torch.tensor(...) 提取为 21x3，
    写入同一个 output_txt_path 中，格式为：
        VarName
        a11 a12 a13
        ...
        a21 a22 a23

        NextVarName
        ...

    参数：
        input_path: 源文件路径（含多行 VarName=torch.tensor([...])）
        output_txt_path: 输出 txt 文件路径
        names_filter: 若提供，仅导出列表中的变量名
        overwrite: True 则覆盖输出文件；False 则追加写（如果文件已存在）

    返回：
        统计 dict：{"saved": n, "skipped": m, "errors": [...], "out_path": "...", "names": [...]}
    """
    mode = "w" if overwrite else "a"
    saved, skipped = 0, 0
    names_written: List[str] = []
    errors: List[str] = []

    with open(input_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # 先把要写的块构造好，最后一次性写入（避免半成品）
    blocks: List[str] = []

    for ln, raw in enumerate(lines, start=1):
        line = raw.strip()
        if not line or line.startswith("#"):
            skipped += 1
            continue

        m = _VAR_RE.match(line)
        if not m:
            skipped += 1
            continue

        varname = m.group(1)
        if names_filter is not None and varname not in names_filter:
            skipped += 1
            continue

        try:
            rhs = line.split("=", 1)[1]
            bracket_block, _, _ = _extract_first_bracket_block(rhs)
        except Exception as e:
            errors.append(f"Line {ln} ({varname}): cannot find bracket block: {e}")
            continue

        try:
            arr = ast.literal_eval(bracket_block)
        except Exception as e:
            errors.append(f"Line {ln} ({varname}): literal_eval failed: {e}")
            continue

        try:
            rows21x3 = _reshape_to_21x3(arr)
        except Exception as e:
            errors.append(f"Line {ln} ({varname}): reshape failed: {e}")
            continue

        # 生成一个块文本：第一行是名称，后面 21 行三列，块间空行
        block_lines = [varname] + [" ".join(str(x) for x in row) for row in rows21x3]
        blocks.append("\n".join(block_lines))
        names_written.append(varname)
        saved += 1

    # 写出
    if blocks:
        os.makedirs(os.path.dirname(os.path.abspath(output_txt_path)) or ".", exist_ok=True)
        with open(output_txt_path, mode, encoding="utf-8") as wf:
            wf.write(("\n\n".join(blocks)).rstrip() + "\n")  # 末尾补换行

    return {
        "saved": saved,
        "skipped": skipped,
        "errors": errors,
        "out_path": os.path.abspath(output_txt_path),
        "names": names_written,
    }


input=r"F:\Projects\diffusers\ProgramData\new_data\param.py"
out=r'F:\Projects\diffusers\ProgramData\new_data\image_smpl.txt'
# from your_module import tensors_to_block_txt
stats = tensors_to_block_txt(
    input_path=input,
    output_txt_path=out,
    names_filter=None,   # 只导出特定名字可传如 ["Avatar_Carol_C1_HP_UI"]
    overwrite=True
)
print(stats)


In [ ]:
"""  把output_vertex.txt处理成一个文件夹，含有每个图片的三维点信息"""
import re
from pathlib import Path

# 行首 header：<name>.(png|jpg|jpeg):
HEADER_RE = re.compile(r'^\s*([^\s:]+?\.(?:png|jpg|jpeg))\s*:', re.IGNORECASE)
TRIPLE_RE = re.compile(
    r'\(\s*([+-]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*,\s*'
    r'([+-]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*,\s*'
    r'([+-]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*\)'
)

def _flush_one(img_name: str, body: str, out_dir: Path):
    if not img_name:
        return
    triples = TRIPLE_RE.findall(body)
    if not triples:
        return
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"{Path(img_name).stem}.txt"  # 文件名为去后缀
    with out_path.open("w", encoding="utf-8") as f:
        f.write(f"{img_name}\n")  # 第一行保留完整图片名（含后缀）
        for x, y, z in triples:
            f.write(f"{x} {y} {z}\n")
    print(f"[saved] {out_path}")

def split_streaming(input_path: str, out_dir: str = "out_txts"):
    out_dir = Path(out_dir)
    current_img = None
    current_body_parts = []

    with open(input_path, "r", encoding="utf-8", errors="ignore") as fh:
        for line in fh:
            m = HEADER_RE.search(line)
            if m:
                # 遇到新的 header：先把旧条目写出
                if current_img is not None:
                    _flush_one(current_img, "".join(current_body_parts), out_dir)
                    current_body_parts = []

                # 启动新条目，正文从冒号后开始
                current_img = m.group(1).strip()
                current_body_parts.append(line[m.end():])
            else:
                # 非 header 行：继续累积到当前条目
                if current_img is not None:
                    current_body_parts.append(line)
                else:
                    # 还没开始条目，丢弃或根据需要处理
                    continue

    # 文件结束后，写出最后一个条目
    if current_img is not None:
        _flush_one(current_img, "".join(current_body_parts), out_dir)

# 调用示例
split_streaming(
    r"F:\Projects\diffusers\ProgramData\new_data\output_vertex\output_vertex.txt",
    out_dir=r"F:\Projects\diffusers\ProgramData\new_data\image\smpl"
)